In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# 1. 数据预处理部分
# FashionMNIST是灰度图像数据集，这里使用适合它的归一化参数进行数据预处理
# 将图像转换为张量并进行归一化，有助于提升模型训练效果
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.2860,), (0.3530,))
])

# 2. 数据集加载部分
# 加载FashionMNIST的训练集，设置根目录、表明是训练集、自动下载以及应用数据预处理转换等参数
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                             download=True, transform=transform)
# 加载FashionMNIST的测试集
testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                            download=True, transform=transform)

# 3. 数据划分部分
# 使用train_test_split将原始训练集划分为新的训练集和验证集，这里采用常见的80%训练集、20%验证集的划分方式
# 可以根据实际需求调整test_size参数来改变划分比例
train_data, val_data = train_test_split(trainset, test_size=0.5, random_state=42)

# 4. 数据加载器创建部分
# 创建训练集数据加载器，设置批量大小、打乱数据顺序以及使用多进程加载（num_workers设为合适值可加快加载速度）
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64,
                                           shuffle=True, num_workers=2)
# 创建验证集数据加载器，不打乱数据顺序
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64,
                                         shuffle=False, num_workers=2)
# 创建测试集数据加载器，同样不打乱数据顺序
test_loader = torch.utils.data.DataLoader(testset, batch_size=64,
                                          shuffle=False, num_workers=2)

# 5. 模型构建与配置部分
# 实例化预训练的ResNet18模型，由于FashionMNIST是单通道灰度图像，需修改conv1层的输入通道数为1
model = resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
# 根据FashionMNIST数据集的类别数量（共10类不同的时尚物品类别）调整全连接层输出维度为10
model.fc = nn.Linear(model.fc.in_features, 10)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 6. 损失函数与优化器定义部分
# 定义交叉熵损失函数，用于衡量多分类任务中模型预测结果与真实标签之间的差异
criterion = nn.CrossEntropyLoss()
# 定义Adam优化器，并设置初始学习率，学习率可根据后续模型训练情况进一步调整优化
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 7. 训练函数定义部分
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        # 将输入数据和目标数据移动到指定的计算设备（GPU或CPU）上
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

# 8. 验证函数定义部分
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    val_loss = running_loss / len(val_loader)
    val_acc = 100. * correct / total
    precision = precision_score(all_targets, all_preds, average='macro')
    recall = recall_score(all_targets, all_preds, average='macro')
    f1 = f1_score(all_targets, all_preds, average='macro')
    conf_matrix = confusion_matrix(all_targets, all_preds)
    return val_loss, val_acc, precision, recall, f1, conf_matrix

# 9. 测试函数定义部分
def test(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    test_acc = 100. * correct / total
    precision = precision_score(all_targets, all_preds, average='macro')
    recall = recall_score(all_targets, all_preds, average='macro')
    f1 = f1_score(all_targets, all_preds, average='macro')
    conf_matrix = confusion_matrix(all_targets, all_preds)
    return test_acc, precision, recall, f1, conf_matrix

# 10. 训练与评估循环部分
epochs = 5
for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, precision, recall, f1, conf_matrix = validate(model, val_loader, criterion, device)
    print(f'Epoch {epoch + 1}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%, '
          f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}')
    print("Confusion Matrix:")
    print(conf_matrix)

test_acc, precision, recall, f1, conf_matrix = test(model, test_loader, device)
print(f"Test Accuracy: {test_acc:.2f}%")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-score: {f1:.4f}")
print("Test Confusion Matrix:")
print(conf_matrix)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1: Train Loss: 0.5766, Train Acc: 79.54%, Val Loss: 0.4160, Val Acc: 84.77%, Precision: 0.8520, Recall: 0.8479, F1-score: 0.8440
Confusion Matrix:
[[2299   19   57  390   21    0  154    0   26    0]
 [   0 2943    5   60    7    0    7    0    1    0]
 [  22    1 2601   66  176    0  146    0   22    0]
 [  17   23   24 2918   19    0   16    0    1    0]
 [   2    3  462  368 1934    0  247    0   17    0]
 [   0    1    2    5    3 2739    3  108   16   94]
 [ 536    5  397  350  168    0 1508    0   42    0]
 [   0    0    0    0    1   44    0 2681    6  223]
 [   3    1    9   22   11    3   51    2 2855    4]
 [   0    0    0    3    1    6    2   60    8 2953]]
Epoch 2: Train Loss: 0.3990, Train Acc: 85.61%, Val Loss: 0.3740, Val Acc: 86.25%, Precision: 0.8698, Recall: 0.8630, F1-score: 0.8576
Confusion Matrix:
[[2691    1   64   71    8    4   73    0   54    0]
 [   7 2926    4   68   11    0    1    0    6    0]
 [  44    1 2766   46   86    1   57    0   33    0]
 [ 1